### SP500 Stock Demo — Notebook 03: Train and Register Model

This notebook trains an XGBoost regressor to predict 3‑month ahead returns from hourly features, logs experiments, and registers the best model.

**What you will do**
- Build `TARGET_PCT_3M` using `lead()` per ticker over time
- Time-based split with safeguards to keep non-empty train/test
- Hyperparameter sweep with explicit experiment tracking (params + metrics)
- Select best model and evaluate train/test metrics
- Register in Snowflake Model Registry and enable explainability

**Outputs**
- Experiment: `SP500_XGB_RET3M`
- Registry model: `XGB_SP500_RET3M` (default set)
- Tables: `FEATURE_SHAP_GLOBAL_TOP`, optionally `FEATURE_IMPORTANCE_SP500`

**Structure**
1) Session + experiment setup
2) Create supervised target
3) Time-based split
4) Train + sweep + log metrics
5) Final selection + evaluation
6) Registry + explainability
7) Optional: list recent runs

**Key functions you will see**
- `lead(col('CLOSE'), horizon)` creates the future value for the target
- `Window.partition_by('TICKER').order_by(col('TS'))` defines per‑ticker ordering
- `Pipeline([...]).fit(train_df)` trains on‑snow; `model.predict(df)` scores on‑snow
- Metrics: `mean_squared_error`, `mean_absolute_percentage_error`, `r2_score`
- Registry: `Registry(...).log_model(...)` and `get_model(name).default = version`


In [ ]:
# 0) Imports and session/context
# - Attach to the active Snowflake session used by this notebook
# - Set warehouse, database, schema for all subsequent operations
# - Initialize Experiment Tracking for logging runs/metrics/params
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, lead, avg, sqrt
from snowflake.snowpark.functions import abs as sp_abs
from snowflake.snowpark.functions import pow as sp_pow
from snowflake.snowpark import Window
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.registry import Registry
from snowflake.ml.experiment.experiment_tracking import ExperimentTracking

session = get_active_session()
session.sql("USE WAREHOUSE DEMO_WH_M").collect()
session.sql("USE DATABASE SP500_STOCK_DEMO").collect()
session.sql("USE SCHEMA DATA").collect()

# Experiment Tracking: all runs in this notebook go under this experiment
exp = ExperimentTracking(session=session)
exp.set_experiment("SP500_XGB_RET3M")

# Quick sanity check the feature table is accessible
session.table('PRICE_FEATURES').limit(5).show()


In [ ]:
snowflake_environment = session.sql('select current_user(), current_version()').collect()
from snowflake.snowpark.version import VERSION
from snowflake.ml import version

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(VERSION[0],VERSION[1],VERSION[2]))
print('Snowflake ML version        : {}.{}.{}'.format(version.VERSION[0],version.VERSION[2],version.VERSION[4]))

### Step 1 — Session and experiment setup

- Set active Snowflake context: warehouse, database, schema
- Initialize Experiment Tracking and select experiment `SP500_XGB_RET3M`
- Sanity check `PRICE_FEATURES` is accessible

Prerequisites:
- Notebook 02 created `SP500_STOCK_DEMO.DATA.PRICE_FEATURES`
- Warehouse sized for model training (e.g., `DEMO_WH_M`)

### Dataset columns (sample shown above)

- `TICKER`: symbol identifier used to partition time windows
- `TS`: timestamp of each hourly observation (ordering key)
- `CLOSE`: hourly close price used to compute the label
- `VOLUME`: hourly traded volume (feature)
- `RET_1`: one‑period return proxy used as a feature
- `SMA_5`, `SMA_20`: simple moving averages over 5 and 20 periods
- `VOL_20`: rolling volatility proxy over 20 periods
- `RSI_PROXY`: simple momentum proxy used as a feature
- `TARGET_PCT_3M`: label — 3‑month ahead percentage return (target)

Key variables (feature intent):
RET_1: very short-term momentum/mean reversion clue from the last period’s return.
SMA_5, SMA_20: short/medium trend signals; their relationship to price or to each other (e.g., crossovers) can signal trend shifts.
VOL_20: rolling dispersion; volatility regimes often change the risk-reward profile and can modulate the meaning of momentum/trend signals.
RSI_PROXY: simplified momentum proxy; momentum can persist, especially over short-to-medium horizons.
TARGET_PCT_3M: the label we predict, defined as future close divided by current close minus one, aligning the task to forward 3‑month return.


In [ ]:
# 1) Build supervised dataset: create 3-month ahead target
# - Source features come from PRICE_FEATURES: TICKER, TS, CLOSE, VOLUME, RET_1, SMA_5, SMA_20, VOL_20, RSI_PROXY, ...
# - Compute FUT_CLOSE as a 378-hour lead of CLOSE, then TARGET_PCT_3M = FUT_CLOSE/CLOSE - 1
# - Drop rows without a future value
hourly = session.table('PRICE_FEATURES')  # Base hourly feature table

horizon_hours = 378  # ~3 months of trading hours
win_order = Window.partition_by('TICKER').order_by(col('TS'))  # Per-ticker chronological order

# Build labeled dataset by adding future close and the 3M-ahead return target
ds = (
    hourly
    # FUT_CLOSE: future close price at +horizon within each ticker's time order
    .with_column('FUT_CLOSE', lead(col('CLOSE'), horizon_hours).over(win_order))
    # TARGET_PCT_3M: percentage return over the horizon
    .with_column('TARGET_PCT_3M', (col('FUT_CLOSE')/col('CLOSE') - 1))
    # Remove helper column; keep clean schema for downstream steps
    .drop('FUT_CLOSE')
    # Keep only rows where the future value exists (label known)
    .filter(col('TARGET_PCT_3M').is_not_null())
)

# Peek at a small sample
ds.limit(5).show()


### Step 2 — Create supervised target

- Goal: turn hourly features into a supervised dataset with a 3‑month ahead target per ticker.

- Choose a cutoff near dataset end; back off 30/60/90/120 days if needed
- Guarantee non-empty train/test while respecting forecasting horizon
- Fallback: approximate 80/20 split by time if necessary

Outputs:
- `train_df`: rows with `TS < cutoff`
- `test_df`: rows with `TS >= cutoff`
- How it’s built:
  - Define per‑ticker chronological window: `Window.partition_by('TICKER').order_by(col('TS'))`
  - Set `horizon_hours = 378` (~3 months of trading hours)
  - Compute future close: `FUT_CLOSE = lead(CLOSE, horizon_hours)` over the window
  - Compute label: `TARGET_PCT_3M = FUT_CLOSE / CLOSE - 1`
  - Drop rows where `FUT_CLOSE` is null (label unknown at horizon)

- Why this design:
  - Long‑horizon target while respecting time order; avoids using any future features
  - Lead computed within each `TICKER` prevents cross‑ticker contamination

- Outputs:
  - `ds`: the original `PRICE_FEATURES` columns plus `TARGET_PCT_3M`
  - A small sample is printed (`ds.limit(5).show()`); see the next markdown cell for column‑by‑column notes

- Notes/assumptions:
  - If trading hours/holidays are irregular, `378` is an approximation; adjust `horizon_hours` as needed
  - Expected input columns include: `TICKER`, `TS`, `CLOSE`, `VOLUME`, `RET_1`, `SMA_5`, `SMA_20`, `VOL_20`, `RSI_PROXY`

In [ ]:
# 2) Time-based train/test split with safeguards
# - Start near the end of the dataset for test split; if empty due to lead horizon, back off in steps
# - Fallback: approximate an 80/20 boundary in time
from datetime import timedelta
from snowflake.snowpark.functions import max as sp_max

# Latest timestamp in the labeled dataset; we will back off from here to define a test window
max_ts_ds = ds.select(sp_max(col('TS')).alias('mx')).collect()[0]['MX']

# Placeholders we will fill once a valid split is found
cutoff = None  # the chosen timestamp boundary
train_df = None  # rows strictly before the cutoff
test_df = None   # rows at or after the cutoff

# Progressive backoff: try several look‑back distances to ensure both splits are non‑empty
for days in [30, 60, 90, 120]:
    try_cutoff = max_ts_ds - timedelta(days=days)  # candidate boundary "days" before the end
    train_try = ds.filter(col('TS') < try_cutoff)  # chronological training set
    test_try = ds.filter(col('TS') >= try_cutoff)  # chronological test set
    # Accept the first candidate that yields non‑empty train and test
    if train_try.count() > 0 and test_try.count() > 0:
        cutoff = try_cutoff
        train_df, test_df = train_try, test_try
        break

# Fallback: if backoff never yielded a valid split, approximate an 80/20 split by time
if cutoff is None:
    from snowflake.snowpark.functions import min as sp_min
    bounds = ds.select(sp_min(col('TS')).alias('mn'), sp_max(col('TS')).alias('mx')).collect()[0]
    mn_ts, mx_ts = bounds['MN'], bounds['MX']  # time range of the dataset
    boundary = mn_ts + (mx_ts - mn_ts) * 0.8    # 80% of the time span from start
    train_df = ds.filter(col('TS') < boundary)
    test_df = ds.filter(col('TS') >= boundary)
    cutoff = boundary

# Quick visibility into the chosen boundary and resulting sizes
print({'cutoff': cutoff})
print('Train rows:', train_df.count())
print('Test rows:', test_df.count())


In [ ]:
# 3) Define feature/label/output columns + basic dataset stats (no training here)
feature_cols = ['RET_1','SMA_5','SMA_20','VOL_20','RSI_PROXY','VOLUME','CLOSE']
label_col = 'TARGET_PCT_3M'
output_col = 'PREDICTED_RETURN'

print({
    'total_rows': ds.count(),
    'train_rows': train_df.count(),
    'test_rows': test_df.count(),
    'num_features': len(feature_cols)
})


### Step 3 — Define Metrics

- Choose a cutoff near dataset end; back off 30/60/90/120 days if needed
- Guarantee non-empty train/test while respecting forecasting horizon
- Fallback: approximate 80/20 split by time if necessary

Outputs:
- `train_df`: rows with `TS < cutoff`
- `test_df`: rows with `TS >= cutoff`

In [ ]:
# 3) Utilities: metrics wrappers and identifier sanitizer (no compute)
# - Compute RMSE/MAPE/R2 using Snowflake ML metrics on Snowpark DataFrames
# - Sanitize run names to valid Snowflake identifiers per docs
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import re


def compute_metrics_quick(df, y_col: str, yhat_col: str) -> dict:
    rmse = mean_squared_error(df=df, y_true_col_names=y_col, y_pred_col_names=yhat_col, squared=False)
    mape = mean_absolute_percentage_error(df=df, y_true_col_names=y_col, y_pred_col_names=yhat_col)
    r2 = r2_score(df=df, y_true_col_name=y_col, y_pred_col_name=yhat_col)
    return {'rmse': rmse, 'mape': mape, 'r2': r2}


def sanitize_identifier(name: str, prefix: str = "RUN_") -> str:
    cleaned = re.sub(r'[^A-Za-z0-9_$]', '_', name)
    if not re.match(r'^[A-Za-z_]', cleaned):
        cleaned = f"{prefix}{cleaned}"
    return cleaned[:255]

### Step 4 — Hyperparameter sweep + experiment logging

- Grid: `max_depth ∈ {4,6}`, `learning_rate ∈ {0.1,0.05}`, `n_estimators ∈ {100,200}`
- For each config:
  - Train `XGBRegressor` pipeline on-snow
  - Predict on test set; compute RMSE, MAPE, R2
  - Log params (with dataset context) and metrics to experiment
- Run names are sanitized to valid SQL identifiers to avoid collisions

In [ ]:
wh = str(session.get_current_warehouse()).strip('"')
print(f"Current warehouse: {wh}")
print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

session.sql(f"alter warehouse {session.get_current_warehouse()} set WAREHOUSE_SIZE = XLARGE WAIT_FOR_COMPLETION = TRUE").collect()

print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

In [ ]:
# 3) Define features and train XGBRegressor (+ compact hyperparam sweep with seaborn plot)
feature_cols = ['RET_1','SMA_5','SMA_20','VOL_20','RSI_PROXY','VOLUME','CLOSE']
label_col = 'TARGET_PCT_3M'
output_col = 'PREDICTED_RETURN'

# Print dataset sizes and feature count before training
print({
    'total_rows': ds.count(),
    'train_rows': train_df.count(),
    'test_rows': test_df.count(),
    'num_features': len(feature_cols)
})

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import re

# Notebook: use Snowflake ML metrics APIs directly (keyword-only args, singular names).
# Note: the server-side stored procedure path uses Snowpark-based metrics due to package availability.

def compute_metrics_quick(df, y_col: str, yhat_col: str) -> dict:
    rmse = mean_squared_error(df=df, y_true_col_names=y_col, y_pred_col_names=yhat_col, squared=False)
    mape = mean_absolute_percentage_error(df=df, y_true_col_names=y_col, y_pred_col_names=yhat_col)
    r2 = r2_score(df=df, y_true_col_name=y_col, y_pred_col_name=yhat_col)
    return {'rmse': rmse, 'mape': mape, 'r2': r2}


def sanitize_identifier(name: str, prefix: str = "RUN_") -> str:
    """Convert an arbitrary string to a valid Snowflake SQL identifier.
    - Replace invalid characters with underscores
    - Ensure it starts with a letter or underscore (else prefix)
    - Truncate to 255 characters
    See: https://docs.snowflake.com/en/sql-reference/identifiers-syntax
    """
    cleaned = re.sub(r'[^A-Za-z0-9_$]', '_', name)
    if not re.match(r'^[A-Za-z_]', cleaned):
        cleaned = f"{prefix}{cleaned}"
    return cleaned[:255]

# Compact grid similar to reference (learning_rate vs mape; hue=n_estimators; facet by max_depth)
max_depth_opts = [4, 6]
learning_rate_opts = [0.1, 0.05]
n_estimators_opts = [100, 200]

rows = []
for depth in max_depth_opts:
    for lr in learning_rate_opts:
        for n in n_estimators_opts:
            display_name = f"xgb_depth={depth}_lr={lr}_n={n}"
            ts = pd.Timestamp.utcnow().strftime('%Y%m%dT%H%M%S')
            run_name = sanitize_identifier(f"{display_name}_{ts}")
            with exp.start_run(run_name):
                exp.log_params({
                    "display_name": display_name,
                    "model": "XGBRegressor",
                    "max_depth": int(depth),
                    "learning_rate": float(lr),
                    "n_estimators": int(n),
                    "subsample": 0.8,
                    "colsample_bytree": 0.8,
                    "feature_cols": feature_cols,
                    "label_col": label_col,
                    "output_col": output_col,
                    "time_split_cutoff": cutoff.isoformat() if cutoff else "auto",
                    "dataset_table": "PRICE_FEATURES",
                    "horizon_hours": int(horizon_hours),
                })

                xgb_tmp = XGBRegressor(
                    n_estimators=n,
                    max_depth=depth,
                    learning_rate=lr,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    input_cols=feature_cols,
                    label_cols=[label_col],
                    output_cols=[output_col],
                    random_state=42,
                )
                model_tmp = Pipeline(steps=[('xgb', xgb_tmp)]).fit(train_df)
                pred_tmp = model_tmp.predict(test_df).select(label_col, output_col)
                m = compute_metrics_quick(pred_tmp, label_col, output_col)
                rows.append({
                    'max_depth': depth,
                    'learning_rate': lr,
                    'n_estimators': n,
                    'rmse': m['rmse'],
                    'mape': m['mape'],
                    'r2': m['r2'],
                })

                exp.log_metrics({
                    "rmse": float(m["rmse"]),
                    "mape": float(m["mape"]),
                    "r2": float(m["r2"]),
                })

hp_df = pd.DataFrame(rows)
# Drop any null metric rows to avoid seaborn errors
hp_df = hp_df.dropna(subset=['mape', 'rmse']).sort_values('mape').reset_index(drop=True)

# Train final model with best params (by RMSE) if available
if not hp_df.empty:
    best = hp_df.sort_values('rmse').iloc[0].to_dict()
    xgb = XGBRegressor(
        n_estimators=int(best['n_estimators']),
        max_depth=int(best['max_depth']),
        learning_rate=float(best['learning_rate']),
        subsample=0.8,
        colsample_bytree=0.8,
        input_cols=feature_cols,
        label_cols=[label_col],
        output_cols=[output_col],
        random_state=42,
    )
    pipe = Pipeline(steps=[('xgb', xgb)])
    model = pipe.fit(train_df)
    print('Model trained with best config (by RMSE):', best)
else:
    # Fall back to baseline params if sweep failed
    best = {'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.05}
    xgb = XGBRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        input_cols=feature_cols,
        label_cols=[label_col],
        output_cols=[output_col],
        random_state=42,
    )
    pipe = Pipeline(steps=[('xgb', xgb)])
    model = pipe.fit(train_df)
    print('Sweep produced no metrics; trained baseline config:', best)

# Plot tuning curve similar to reference (learning_rate vs MAPE) if data available
if not hp_df.empty:
    sns.set_context('notebook', font_scale=0.9)
    # Use FacetGrid-like relplot to match style
    g = sns.relplot(
        data=hp_df,
        x='learning_rate', y='mape', hue='n_estimators', col='max_depth',
        kind='line', marker='o', height=4, aspect=1.2
    )
    g.set_titles('max_depth = {col_name}')
    g.set_axis_labels('learning_rate', 'MAPE')
else:
    print('Skipping tuning plot: no sweep results')

### Step 4 (cont.) — Consolidate sweep and train best model

- Build DataFrame of sweep results and drop null rows
- Select best by RMSE and refit full model on `train_df` (fallback to baseline if no results)
- Optionally plot tuning curve (learning_rate vs MAPE; hue = `n_estimators`; facet = `max_depth`)


In [ ]:
# 4) Evaluate metrics (RMSE, MAPE, R2) using robust helper
train_pred = model.predict(train_df).select(label_col, output_col)
test_pred = model.predict(test_df).select(label_col, output_col)

train_metrics = compute_metrics_quick(train_pred, label_col, output_col)
test_metrics = compute_metrics_quick(test_pred, label_col, output_col)

print({'train': train_metrics, 'test': test_metrics})

# Log final evaluation and selection as a run for traceability
ts_final = pd.Timestamp.utcnow().strftime('%Y%m%dT%H%M%S')
with exp.start_run(f"final_best_selection_{ts_final}"):
    exp.log_params({
        "selected_by": "rmse",
        "best_max_depth": int(best["max_depth"]) if isinstance(best["max_depth"], (int, float, str)) else best["max_depth"],
        "best_learning_rate": float(best["learning_rate"]),
        "best_n_estimators": int(best["n_estimators"]),
        "feature_cols": feature_cols,
        "label_col": label_col,
        "output_col": output_col,
        "time_split_cutoff": cutoff.isoformat() if cutoff else "auto",
        "dataset_table": "PRICE_FEATURES",
        "horizon_hours": int(horizon_hours),
    })

    exp.log_metrics({
        "train_rmse": float(train_metrics["rmse"]),
        "train_mape": float(train_metrics["mape"]),
        "train_r2": float(train_metrics["r2"]),
        "test_rmse": float(test_metrics["rmse"]),
        "test_mape": float(test_metrics["mape"]),
        "test_r2": float(test_metrics["r2"]),
    })


In [ ]:
# 4c) Diagnostic plots: predicted vs actual, residuals, calibration, and a sample ticker timeseries

# Score test set keeping identifiers for plotting
scored_sp = model.predict(test_df).select('TICKER', 'TS', label_col, output_col)

# Bring a manageable sample to pandas for plotting (adjust if needed)
max_rows = 50000
scored_pd = scored_sp.limit(max_rows).to_pandas()
scored_pd = scored_pd.rename(columns={label_col: 'y_true', output_col: 'y_pred'})
scored_pd['residual'] = scored_pd['y_pred'] - scored_pd['y_true']

# Basic 2x2 diagnostics
sns.set_context('notebook', font_scale=0.9)
fig, axes = plt.subplots(2, 2, figsize=(12, 9))

# 1) Predicted vs Actual
ax = axes[0, 0]
ax.scatter(scored_pd['y_true'], scored_pd['y_pred'], s=8, alpha=0.4)
lims = [np.nanmin([scored_pd['y_true'], scored_pd['y_pred']]), np.nanmax([scored_pd['y_true'], scored_pd['y_pred']])]
ax.plot(lims, lims, 'r--', linewidth=1)
ax.set_title('Predicted vs Actual (test)')
ax.set_xlabel('Actual (y_true)')
ax.set_ylabel('Predicted (y_pred)')

# 2) Residuals histogram
ax = axes[0, 1]
sns.histplot(scored_pd['residual'], bins=40, kde=True, ax=ax)
ax.set_title('Residuals distribution (y_pred - y_true)')
ax.set_xlabel('Residual')

# 3) Residuals vs Fitted
ax = axes[1, 0]
ax.scatter(scored_pd['y_pred'], scored_pd['residual'], s=8, alpha=0.4)
ax.axhline(0.0, color='r', linestyle='--', linewidth=1)
ax.set_title('Residuals vs Fitted')
ax.set_xlabel('Predicted (y_pred)')
ax.set_ylabel('Residual')

# 4) Calibration-style: bin by predicted and compare means
ax = axes[1, 1]
try:
    scored_pd['pred_bin'] = pd.qcut(scored_pd['y_pred'], q=10, duplicates='drop')
    cal = scored_pd.groupby('pred_bin').agg(mean_pred=('y_pred','mean'), mean_true=('y_true','mean')).reset_index(drop=True)
    ax.plot(cal['mean_pred'], cal['mean_true'], marker='o')
    lims = [np.nanmin([cal['mean_pred'], cal['mean_true']]), np.nanmax([cal['mean_pred'], cal['mean_true']])]
    ax.plot(lims, lims, 'r--', linewidth=1)
    ax.set_title('Calibration (bin-mean actual vs predicted)')
    ax.set_xlabel('Mean predicted (per bin)')
    ax.set_ylabel('Mean actual (per bin)')
except Exception:
    ax.text(0.5, 0.5, 'Calibration plot unavailable (insufficient variance)', ha='center')
    ax.axis('off')

plt.tight_layout()
plt.show()

# Time series for a representative ticker (largest test coverage)
if 'TICKER' in scored_pd.columns and 'TS' in scored_pd.columns:
    top_ticker = scored_pd['TICKER'].value_counts().index[0]
    ts_pd = scored_sp.filter(col('TICKER') == top_ticker).order_by(col('TS')).limit(2000).to_pandas()
    ts_pd = ts_pd.rename(columns={label_col: 'y_true', output_col: 'y_pred'})
    plt.figure(figsize=(12, 4))
    plt.plot(ts_pd['TS'], ts_pd['y_true'], label='Actual', linewidth=1)
    plt.plot(ts_pd['TS'], ts_pd['y_pred'], label='Predicted', linewidth=1)
    plt.title(f'Actual vs Predicted over time — {top_ticker}')
    plt.xlabel('Time (TS)')
    plt.ylabel('Return (3M ahead)')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Optional: residuals by ticker (top 12 by count)
if 'TICKER' in scored_pd.columns:
    top_tickers = scored_pd['TICKER'].value_counts().head(12).index
    sub = scored_pd[scored_pd['TICKER'].isin(top_tickers)]
    plt.figure(figsize=(12, 5))
    sns.boxplot(data=sub, x='TICKER', y='residual')
    plt.title('Residuals by Ticker (top 12 by test rows)')
    plt.xlabel('Ticker')
    plt.ylabel('Residual (y_pred - y_true)')
    plt.xticks(rotation=30)
    plt.tight_layout()
    plt.show()


### Diagnostic plots — interpretation and next steps

- Predicted vs Actual (test):
  - Expectation: tight cloud around 45°; unbiased scale.
  - Observed: predictions compressed near 0; underprediction for positive actuals. Indicates weak signal and shrinkage to mean.

- Residuals distribution:
  - Expectation: centered around 0, roughly symmetric.
  - Observed: long negative tail; asymmetric. Systematic underprediction when actuals are large.

- Residuals vs Fitted:
  - Expectation: no structure; constant variance around 0.
  - Observed: banding/heteroskedasticity. Missing regime/ticker structure.

- Calibration (bin-mean actual vs predicted):
  - Expectation: near 45° line.
  - Observed: off-diagonal and irregular. Poor calibration/scale.

- Time series (sample ticker):
  - Expectation: predicted co-moves with actual and captures direction.
  - Observed: predicted nearly flat near 0 while actuals vary substantially.

- Residuals by ticker:
  - Expectation: medians near 0, similar spread.
  - Observed: ticker-specific bias and different spreads.

Metrics snapshot (best config):
- Train: rmse=0.1653, mape=1.8218, r2=0.0735
- Test: rmse=0.1401, mape=2.1695, r2=−0.0714
- Takeaway: negative test R² and poor plots imply the model underperforms a constant baseline and is poorly calibrated; predictions are overly shrunk to zero.

Actionable improvements:
- Features: add market/sector context (SPY/sector returns), multi-horizon momentum (RET_5/RET_20), SMA/EMA ratios, rolling beta, volatility regime (VIX), cross-sectional ranks.
- Target: consider log-return, winsorization, or reframing to sign classification/ranking; MAPE is unstable for near-zero targets.
- Training: time-series CV, wider hyperparam search with early stopping, longer n_estimators, deeper trees; expand data horizon.
- Post-model: revisit calibration after achieving non-negative R².


### Step 5 — Final selection and evaluation

- Re-score train/test using the selected model
- Compute and print RMSE, MAPE, R2 for both splits
- Log `final_best_selection` run with:
  - Selected hyperparameters
  - Dataset context (features, label, cutoff, table, horizon)
  - Final metrics (`train_*`, `test_*`)

Tip: Run names and identifiers are sanitized (letters/digits/`_`/`$`, start with letter/underscore, max 255 chars).

In [ ]:
# 5) Register in Model Registry (enable Snowflake explainability)
reg = Registry(session=session, database_name='SP500_STOCK_DEMO', schema_name='DATA')
model_name = 'XGB_SP500_RET3M'

models_df = reg.show_models()
if models_df.empty or model_name not in models_df['name'].to_list():
    version = 'V_1'
else:
    import ast, builtins
    max_v = builtins.max([int(v.split('_')[-1]) for v in ast.literal_eval(models_df.loc[models_df['name']==model_name,'versions'].values[0])])
    version = f'V_{max_v+1}'

# Provide background data to enable SHAP explainability in the Model Registry
bg_pd = train_df.select(feature_cols).limit(1000).to_pandas()

mv = reg.log_model(
    model,
    model_name=model_name,
    version_name=version,
    conda_dependencies=['snowflake-ml-python'],
    comment='XGBRegressor predicting 3-month forward return from hourly features',
    metrics={
        'train_r2': train_metrics['r2'],
        'test_r2': test_metrics['r2'],
        'train_rmse': train_metrics['rmse'],
        'test_rmse': test_metrics['rmse'],
        'train_mape': train_metrics['mape'],
        'test_mape': test_metrics['mape'],
    },
    sample_input_data=bg_pd,
    options={'relax_version': False, 'enable_explainability': True}
)
reg.get_model(model_name).default = version

# Log registry pointers into the final run context
ts_reg = pd.Timestamp.utcnow().strftime('%Y%m%dT%H%M%S')
with exp.start_run(f"final_best_selection_regref_{ts_reg}"):
    exp.log_params({
        "registry_model_name": model_name,
        "registry_version": version,
    })

print({'model_name': model_name, 'version': version})


### Step 6 — Model Registry and experiment linkage

- Register model in Snowflake Model Registry and set default version
- Provide background sample to enable explainability (`explain()`): ~1000 training rows of features
- Log registry pointers back to the experiment (`registry_model_name`, `registry_version`) for traceability

Versioning:
- If the model exists, increment numeric suffix (e.g., `V_2`, `V_3`)


In [ ]:
# 6) Snowflake explainability: compute SHAP-based global feature importance + seaborn bar plot
# explain() availability was enabled during logging (cell 5) using background data.
from snowflake.snowpark.functions import abs as sp_abs, col as sp_col
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

reg = Registry(session=session, database_name='SP500_STOCK_DEMO', schema_name='DATA')
mv_explain = reg.get_model('XGB_SP500_RET3M').default

# Take a small sample of inputs (features only) for explanations
sample_sp = ds.select(*feature_cols).limit(200)

# Run SHAP explain through Model Registry (Snowflake-built explainability)
explanations_sp = mv_explain.run(sample_sp, function_name='explain')

# Aggregate mean absolute SHAP per feature
ex_pd = explanations_sp.to_pandas()
cols = list(ex_pd.columns)
shap_map = {}
for f in feature_cols:
    if f in cols:
        shap_map[f] = f
    elif f"SHAP_{f}" in cols:
        shap_map[f] = f"SHAP_{f}"

imp_rows = []
for f, c in shap_map.items():
    imp = float(pd.Series(ex_pd[c]).abs().mean())
    imp_rows.append({'feature': f, 'mean_abs_shap': imp})
imp_df = pd.DataFrame(imp_rows).sort_values('mean_abs_shap', ascending=False)

# Persist and plot
session.create_dataframe(imp_df).write.save_as_table('FEATURE_SHAP_GLOBAL_TOP', mode='overwrite')

sns.set_context('notebook', font_scale=0.9)
plt.figure(figsize=(8, 4))
sns.barplot(data=imp_df, x='mean_abs_shap', y='feature', orient='h')
plt.title('Global feature importance (mean |SHAP|)')
plt.xlabel('Mean |SHAP|')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()
print(imp_df.head(10))


In [ ]:
# 6b) Explainability: feature importances (from sklearn view) — optional quick view
sk_pipe = model.to_sklearn()
# Find xgb step
xgb_step = None
for name, step in sk_pipe.named_steps.items():
    if 'xgb' in name.lower():
        xgb_step = step
        break

if xgb_step is not None and hasattr(xgb_step, 'feature_importances_'):
    import pandas as pd
    fi = pd.DataFrame({
        'feature': feature_cols,
        'importance': xgb_step.feature_importances_.tolist()[:len(feature_cols)]
    }).sort_values('importance', ascending=False)
    fi_sp = session.create_dataframe(fi)
    fi_sp.write.save_as_table('FEATURE_IMPORTANCE_SP500', mode='overwrite')
    print('Saved FEATURE_IMPORTANCE_SP500')
else:
    print('Feature importances not available from sklearn object.')


In [ ]:
wh = str(session.get_current_warehouse()).strip('"')
print(f"Current warehouse: {wh}")
print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

session.sql(f"alter warehouse {session.get_current_warehouse()} set WAREHOUSE_SIZE = SMALL WAIT_FOR_COMPLETION = TRUE").collect()

print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())